# Adding ideologies of judges

In [85]:
import pandas as pd

In [86]:
WORKDIR = "/data/WorkData/ideology_from_audio/RESULTS"

In [89]:
ls $WORKDIR

contriblawyers.csv  justices.csv     raw_wave_word.csv
ideodefined.txt     listpeople.txt*  WordAudio/
ideologyfinal.txt   listwords.txt*   wordssyllablesformants.txt*


In [90]:
def strip_beginning_uppercase(string):
    i = 0
    while(string[i] == str.upper(string[i])):
        i += 1
    return string[i-1:].lower()

In [122]:
# Load justice dataset and normalize posterior mean Quinn-Martin score
justices = pd.read_csv(WORKDIR + "/justices.csv")
justices = justices.drop(['justice','post_sd', 'post_med', 'post_025', 'post_975'], axis = 1)
justices.post_mn = (justices.post_mn - (-8) )/ 16
# Keep only relevant years
justices = justices[justices.term >= 1998]
justices = justices[justices.term <= 2013].reset_index().drop('index', axis = 1)

# Keep only last uppercase letter
justices.justiceName = justices.justiceName.apply(strip_beginning_uppercase)
justices.loc[justices['justiceName'] == 'connor', 'justiceName'] = 'oconnor'

In [126]:
av_MQ_scores = justices.groupby('justiceName').mean().drop('term', axis=1)
av_MQ_scores

,post_mn
justiceName,
alito,0.608639
breyer,0.423145
ginsburg,0.389719
kagan,0.406078
kennedy,0.537977
oconnor,0.526586
rehnquist,0.595339
roberts,0.579931
scalia,0.662453


In [127]:
ideology = pd.read_csv(WORKDIR + "/ideologyfinal.txt", sep=",", names = ['first', 'last', 'ideology'])
ideology.head()

,first,last,ideology
0,aaron,weisman,undefined
1,aaron,panner,0.0
2,aj,kramer,undefined
3,alan,gura,1.0
4,alexander,ross,undefined


In [131]:
for last_name, row in av_MQ_scores.iterrows():
    ideology.loc[ideology['last'] == last_name, 'ideology'] = row.post_mn

In [133]:
ideology.head()

,first,last,ideology
0,aaron,weisman,undefined
1,aaron,panner,0.0
2,aj,kramer,undefined
3,alan,gura,1.0
4,alexander,ross,undefined


In [138]:
ideology.to_csv(WORKDIR + "/ideologyfinal.txt", sep = ",")